# Carga y limpieza de los datos
[atrás](./)

# Índice

1. [Importar librerias](##1.-Importar-librerias)  
2. [Funciones auxiliares](##2.-Funciones-auxiliares)  
3. [Carga del Dataset](##3.-Carga-del-Dataset)  
4. [Limpieza de los datos](##4.-Limpieza-de-los-datos)  
   - [4.1 Estudio de las columnas](###4.1-Estudio-de-las-columnas)
   - [4.2 Tratamiento de los duplicados y de los nulos](###4.2-Tratamiento-de-los-duplicados-y-de-los-nulos)
   - [4.3 Tratamiento de las variables categóricas](###4.3-Tratamiento-de-las-variables-categoricas)
      - [4.3.1 Tratamiento de las variables categóricas binarias](####4.3.1-Tratamiento-de-las-variables-categoricas-binarias)
      - [4.3.2 Tratamiento de las variables categóricas no binarias](####4.3.2-Tratamiento-de-las-variables-categoricas-no-binarias)

5. [Correlacion](##5.-Correlacion)  
6. [Codificacion One-Hot](##6.-Codificacion-One-Hot)  
7. [Create Test](##7.-Create-Test)  
8. [Exportar CSV](##8.-Exportar-CSV)



## 1. Importar librerias

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder

ModuleNotFoundError: No module named 'seaborn'

## 2. Funciones auxiliares

In [ ]:
def extract_month(date, mode=0):
    if pd.isna(date):
        return mode
    return datetime.strptime(date, '%d-%b-%y').month

In [ ]:
def month_to_quarter(month):
    if month == 0:
        return 0
    return np.ceil(month / 3).astype(int)

## 3. Carga del Dataset

In [ ]:
url = "data/train.csv" 
data = pd.read_csv(url)
df = data.copy()

Configuramos Pandas para mostrar todas las columnas de un DataFrame al imprimirlo, y luego mostramos las primeras filas del DataFrame df.

In [ ]:
pd.set_option('display.max_columns', None)
df.head()

Con *Info* desplegamos la información del dataframe para saber con que datos trabajaremos en cada una de las columnas

In [ ]:
df.info

Gracias a *shape* podemos saber las dimensiones del dataframe, de esta maenra podemos saber la cantidad de filas que tenemos en un incio y la cantidad de columnas con las que trabajaremos

In [ ]:
df.shape

Visualización del nombre de todas las columnas para plantear la limpieza

In [ ]:
df.columns


Visualización del estadisticos de las columnas numericas para plantear la limpieza

In [ ]:
df.select_dtypes(include=['int64', 'float64']).describe()

## 4. Limpieza de los datos

### 4.1 Estudio de las columnas

| **Field Name** | **Data Type** | **Description** |
| --- |  --- |  --- |
| **id** | Text | Identifier of the data instance |
| **LoanNr\_ChkDgt** | Text | Identifier of the loan petition |
| **Name** | Text | Borrower name |
| **City** | Text | Borrower city |
| **State** | Text | Borrower state |
| **Bank** | Text | Bank name |
| **BankState** | Text | Bank state |
| **ApprovalDate** | Date/Time | Date SBA commitment issued |
| **ApprovalFY** | Text | Fiscal year of commitment |
| **NoEmp** | Number | Number of business employees |
| **NewExist** | Text | 1 = Existing business, 2 = New business |
| **CreateJob** | Number | Number of jobs created |
| **RetainedJob** | Number | Number of jobs retained |
| **FranchiseCode** | Text | Franchise code, (00000 or 00001) = No franchise |
| **UrbanRural** | Text | 1 = Urban, 2 = rural, 0 = undefined |
| **RevLineCr** | Text | Revolving line of credit: Y = Yes, N = No |
| **LowDoc** | Text | LowDoc Loan Program: Y = Yes, N = No |
| **ChgOffDate** | Date/Time | The date when a loan is declared to be in default (no en Dataframe)|
| **DisbursementDate** | Date/Time | Disbursement date |
| **DisbursementGross** | Currency | Amount disbursed |
| **BalanceGross** | Currency | Gross amount outstanding |
| **Accept** | Text | Loan approval status. 0 = not approved, 1 = approved |


Visualización de los tipos de cada columna

In [ ]:
df.dtypes

Visualizacón de valores distintos que tienen cada columna

In [ ]:
# Obtener el número de valores distintos de cada columna
valores_distintos = {columna: df[columna].nunique() for columna in df.columns}

# Mostrar el número de valores distintos de cada columna
for columna, num_valores in valores_distintos.items():
    print(f"Columna: {columna}, Valores distintos: {num_valores}")

Creación de diccionario para poder ver de manera más compacta los diferentes valores que componen cada columna

In [ ]:
# Crear un diccionario vacío para almacenar los resultados
conteo_valores = {}

# Iterar sobre cada columna del dataframe
for columna in df.columns:
    # Usar value_counts() para obtener los conteos y convertir a diccionario
    conteo = df[columna].value_counts().to_dict()
    conteo_valores[columna] = conteo

Extraccíon de los datos de cada columna (Está comnetado ya que solo se visuliza de manera adecuada en vscode)

In [ ]:
# print("Bank              ->",conteo_valores["Bank"])
# print("BankState         ->",conteo_valores["BankState"])
# print("ApprovalDate      ->",conteo_valores["ApprovalDate"])
# print("ApprovalFY        ->",conteo_valores["ApprovalFY"])
# print("NewExist          ->",conteo_valores["NewExist"])
# print("RetainedJob       ->",conteo_valores["RetainedJob"])
# print("FranchiseCode     ->",conteo_valores["FranchiseCode"])
# print("UrbanRural        ->",conteo_valores["UrbanRural"])
# print("LowDoc            ->",conteo_valores["LowDoc"])
# print("RevLineCr         ->",conteo_valores["RevLineCr"])
# print("DisbursementDate  ->",conteo_valores["DisbursementDate"])
# print("DisbursementGross ->",conteo_valores["DisbursementGross"])
# print("BalanceGross      ->",conteo_valores["BalanceGross"])
# print("Accept            ->",conteo_valores["Accept"])

Gracias a todos la info extraida anteriormente podemos eliminar las columnas que no hacen falta, agrupar las columnas en función de como deberemos tratar los datos y plantear los procesos que sufrirán cada una de ellas.

    - *Id*, *LoanNr_ChkDgt*, *Name* y *City* : no merecen la pena formalizarlos.
    - *State* : solo puede tener un valor por lo que no lo usaremos.
    - *Bank* : categorizar en 4 grupos, 3 bankos mas valores y 1 para otros
    - *BankState* : categorizar en 5 grupos, 4 estados con mas  mas valores y 1 para otros
    - *ApprovalDate*, *DisbursementDate* :fechas podemos sacar el mes y hacerlos trimestrales para hacerlos categorias.
    - *ApprovalFY* : Agrupar en periodos de 10 años para hacerlo categorías
    - *NoEmp*, *CreateJob*, *RetainedJob*: Agrupar en rangos, haremos histograma
    - *NewExist* : Pasar a int
    - *FranchiseCode* : Agrupar en si es franquicia o no 
    - *UrbanRural*  : Quitar undefined para que haya solo 1 o 0
    - *RevLineCr*, *LowDoc*: pasar a binario Y o N, 1 o 0
    - *DisbursementGross* : Agrupar en cantidad, histograma
    - *BalanceGross* : No vale para nada este valor, todos son 0, y hay 2 valores que no dan info
    - *Accept* : ya esta estandarizado

Grupos:
1. **featuresUseless**              
    'id', 'LoanNr_ChkDgt', 'Name', 'City', 'State', 'BalanceGross'
2. **Binarios**  
    'NewExist', 'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc', 'Accept' 
3. **Fechas**  
    'ApprovalDate', 'DisbursementDate'
4. **Otros**    
    'Bank', 'BankState', 'ApprovalFY', 'NoEmp', 'CreateJob', 'RetainedJob', 'DisbursementGross'

Array de columnas que no sirven

In [ ]:
featuresUseless = ['id', 'LoanNr_ChkDgt', 'Name', 'City', 'State', 'BalanceGross']

Creación del data frame ***dfc*** a partir del dataframe original ***df*** sin los valores que no tienen nigún tipo de utilidad

In [ ]:
dfc = df.drop(columns=featuresUseless)
dfc.head()

### 4.2 Tratamiento de los duplicados y de los nulos

Comprobación de duplicados

In [ ]:
dfc.duplicated().any() 

Al haber duplicados los eliminaremos al no aportan nigún valor 

In [ ]:
dfc.drop_duplicates(inplace=True)

Verificación de que se han eliminado correctamente

In [ ]:
dfc.duplicated().any() 

Comporbación de si existen nulos y en que columnas se encuentran

In [ ]:
dfc.isnull().sum() 

Analizar a qué valor Accept hacen referencia, para determinar que tan significativos son estos datos.

In [ ]:
dfc_null_accept = dfc[dfc.isnull().any(axis=1)]['Accept']
print(dfc_null_accept.value_counts())

Al ser la mayoría Accept 1 y al ser solamente 217 caso de null podemos eliminarlos ya que no son muy significativos.

In [ ]:
dfc = dfc.dropna()

Verficación de que se han eliminado correctamente

In [ ]:
dfc.isnull().sum() 

### 4.3 Tratamiento de las variables categoricas

Gracias al pequeño estudio que se ha realizado en el apartado [1.4.1](##141-estudio-de-las-columnas) podemos empezar a tratar las diferentes variables categóricas.

Visualización de las columnas a tratar

In [ ]:
dfc.dtypes

#### 4.3.1 Tratamiento de las variables categoricas binarias

##### - NewExist

Comprobación de todos los valores de la columna *NewExist*

In [ ]:
dfc["NewExist"].value_counts()

Al tener valores que no crresponden con 1 (Existing business) o 2 (New Business) comprobaremos su valor a partir de ver su distribucíón con la columna Accept

In [ ]:
dfc_null_accept = dfc[dfc['NewExist'] == 0.0]['Accept']
dfc_null_accept.value_counts()

Al ser 12 casos y solamente uno de ellos con valor *Accept* 0 podemos eliminarlos

In [ ]:
# Eliminar las filas donde 'NewExist' es 0.0
dfc = dfc[dfc['NewExist'] != 0.0]

Codificación a Binario los valores de las columna, se ha decido que: **1** *Existing business* y **2** *New business*

In [ ]:
# 1 = Existing business, 2 = New business

dfc['NewExist_Binary'] = dfc['NewExist'].replace({
    1.0: 1,   # Existing -> 1
    2.0: 0,   # New -> 0
}).astype('int64')

Comprobación del proceso

In [ ]:
dfc['NewExist_Binary'].value_counts()

Eliminar columna original

In [ ]:
dfc = dfc.drop(columns=['NewExist'])

##### - FranchiseCode

Comprobación de todos los valores de la columna *FranchiseCode*


In [ ]:
dfc['FranchiseCode'].value_counts()

La columna "franquicia" se convertirá a binario para representar de forma numérica si un establecimiento es franquicia o no.
- **1** o **0**: Indica que el establecimiento es una franquicia.
- ***otros***: Indica que el establecimiento no es una franquicia.


In [ ]:
# Franchise code, (00000 or 00001) = No franchise
dfc['Franchise_Binary'] = np.where(dfc['FranchiseCode'].isin([0, 1]), 0, 1)

Comprobación del proceso

In [ ]:
dfc["Franchise_Binary"].value_counts()

Eliminar columna orginal

In [ ]:
dfc = dfc.drop(columns=['FranchiseCode'])

##### - UrbanRural

Comprobación de todos los valores de la columna *UrbanRural*

In [ ]:
dfc['UrbanRural'].value_counts()

Al haber muchos con el valor **0** (*Undefined*) no podemos simplemente eliminar el dato, por lo que lo transformaremos en valor que más probable pueda ser (la moda) 

In [ ]:
moda = dfc['UrbanRural'].mode()[0]

# Mapeo: 1.0 -> 1 (Existing), 2.0 -> 0 (New), 0.0 -> decidir (ver abajo)
dfc['UrbanRural_Binary'] = dfc['UrbanRural'].replace({
    0: moda    
})


Codificación a Binario de los valores, asingando **1** a ser urbano y **0** a ser rural

In [ ]:
dfc['UrbanRural_Binary'] = dfc['UrbanRural_Binary'].replace({
    1: 1,
    2: 0    
})

Comprobación del proceso

In [ ]:
dfc["UrbanRural_Binary"].value_counts()

Eliminar columna orginal

In [ ]:
dfc = dfc.drop(columns=['UrbanRural'])

##### - RevLineCr

Comprobación de todos los valores de la columna *RevLineCr*

In [ ]:
dfc['RevLineCr'].value_counts()

Por lo tanto, categorizaremos: 
- Yes:    **'Y'** y **'T'**
- No:     **'N'** y **'0'**

El caso de **' ` '** al ser solo uno, se ha decidido eliminar

In [ ]:
# Mapeo de categorías
category_map = {'Y': 1, 'T': 1, 'N': 0, '0': 0}

# Eliminar el valor '`'
dfc = dfc[dfc['RevLineCr'] != '`'] 

# Aplicar la categorización
dfc['RevLineCr_Binary'] = dfc['RevLineCr'].map(category_map)


Comprobación del proceso

In [ ]:
dfc['RevLineCr_Binary'].value_counts()

Eliminar columna orginal

In [ ]:
dfc = dfc.drop(columns=['RevLineCr'])

##### - LowDoc

Comprobación de todos los valores de la columna *LowDoc*

In [ ]:
dfc['LowDoc'].value_counts()

Por lo tanto, categorizaremos: 
- Yes:    **'Y'**
- No:     **'N'** y **'0'**

Los casos de **'A'**,**'S'**,**'R'**,**'C'** al ser pocos y no quedar claro que podrían significar, se ha decidido eliminar

In [ ]:
# Eliminar valores 'A', 'S', 'R', 'C'
dfc = dfc[dfc['LowDoc'].isin(['Y', 'N', '0'])]  

# Reemplazar '0' por 'N' para unificar la categoría "No"
dfc['LowDoc'] = dfc['LowDoc'].replace('0', 'N')

# Codificacion 'Y' a 1 y 'N' a 0
dfc['LowDoc_Binary'] = dfc['LowDoc'].map({'Y': 1, 'N': 0})

Comprobación del proceso

In [ ]:
dfc['LowDoc_Binary'].value_counts()

Eliminar columna orginal

In [ ]:
dfc = dfc.drop(columns=['LowDoc'])

#### 4.3.2 Tratamiento de las variables categoricas no binarias

##### - ApprovalDate

Comprobación de todos los valores de la columna *ApprovalDate*

In [ ]:
dfc['ApprovalDate'].value_counts()


Los meses al estar en fomrato dd-mm-yy no tiene ninguna información como tal, por lo que se ha decido extraer el mes

In [ ]:
dfc['ApprovalDate'] = dfc['ApprovalDate'].apply(extract_month)

Comprobación del procedimiento

In [ ]:
dfc['ApprovalDate'].value_counts().sum()

Creación de una nueva categoría a partir de los meses, agrupándolos en trimestres.

In [ ]:
dfc['ApprovalDate_quarter'] = dfc['ApprovalDate'].apply(month_to_quarter)


Comprobación el proceso

In [ ]:
dfc['ApprovalDate_quarter'].value_counts()

##### - DisbursementDate

Comprobación de todos los valores de la columna *DisbursementDate*

In [ ]:
dfc["DisbursementDate"].value_counts()

Los meses al estar en fomrato dd-mm-yy no tiene ninguna información como tal, por lo que se ha decido extraer el mes

In [ ]:
dfc['DisbursementDate'] = dfc['DisbursementDate'].apply(extract_month)

Comprobación el proceso

In [ ]:
dfc['DisbursementDate'].value_counts()

Creación de una nueva categoría a partir de los meses, agrupándolos en trimestres.

In [ ]:
dfc['DisbursementDate_quarter'] = dfc['DisbursementDate'].apply(month_to_quarter)


Comprobación el proceso

In [ ]:
dfc['DisbursementDate_quarter'].value_counts()

##### - Bank 

Comprobación de todos los valores de la columna *Bank*


In [ ]:
dfc['Bank'].value_counts()

Seleccionaremos los ***Bank*** que superen un umbral específico en proporción a la cantidad que aparecen, y agruparemos el resto en la categoría 'otros'.

In [ ]:
# Contar ocurrencias de cada banco
bank_counts = dfc["Bank"].value_counts()
total = bank_counts.sum()

# Ajusta el umbral para cambiar la cantidad de categorías
threshold = 0.01

# Seleccionar los bancos que cumplen con el umbral
selected_banks = bank_counts[bank_counts / total >= threshold].index

# Crear la nueva columna categorizada en el DataFrame
dfc["Bank_Categorized"] = dfc["Bank"].apply(lambda x: x if x in selected_banks else "Otros")

# Mostrar el conteo por categoría
dfc["Bank_Categorized"].value_counts()


Codificaremos los diferentes ***Bank_Categorized*** para facilitar el entrenamiento en determinados modelos.

In [ ]:
dfc["Bank_Categorized_cod"] = dfc["Bank_Categorized"].astype('category').cat.codes.astype('int64')
dfc["Bank_Categorized_cod"].value_counts()


Eliminar columna orginal

In [ ]:
dfc = dfc.drop(columns=['Bank'])

##### - BankState

Comprobación de todos los valores de la columna *BankState*

In [ ]:
dfc['BankState'].value_counts()

Seleccionaremos los ***BankState*** que superen un umbral específico en proporción a la cantidad que aparecen, y agruparemos el resto en la categoría 'otros'.

In [ ]:
# Contar ocurrencias de cada banco
bank_counts = dfc["BankState"].value_counts()

def categorize_banksState(threshold=0.01):
    """
    Agrupa los estados de los bancos en categorías según un umbral de porcentaje.
    """
    total = bank_counts.sum()
    selected_banks = bank_counts[bank_counts / total >= threshold].index
    
    dfc["BankState_Categorized"] = df["BankState"].apply(lambda x: x if x in selected_banks else "Otros")
    return dfc

# Ajusta el umbral para cambiar la cantidad de categorías
threshold = 0.05
# 1% de presencia en el dataset
dfc = categorize_banksState(threshold)

# Mostrar resultado
print(dfc["BankState_Categorized"].value_counts())

Codificaremos los diferentes ***BankState_Categorized*** para facilitar el entrenamiento en determinados modelos.

In [ ]:
dfc["BankState_Categorized_cod"] = dfc["BankState_Categorized"].astype('category').cat.codes.astype('int64')   
print(dfc["BankState_Categorized_cod"].value_counts())

Eliminar columna orginal

In [ ]:
dfc = dfc.drop(columns=['BankState'])

##### - ApprovalFY

Comprobación de todos los valores de la columna *ApprovalFY*

In [ ]:
dfc['ApprovalFY'].value_counts()

Limpiamos los datos para garantizar que no existan errores de formato.

In [ ]:
# Limpiar y pasar a entero
dfc['ApprovalFY'] = dfc['ApprovalFY'].astype(str).str.extract(r'(\d{4})')  # Extraer solo los años
dfc['ApprovalFY'] = dfc['ApprovalFY'].astype(int)  # Convertir a entero


Agrupar las diferentes fechas en grupos de 5 años


In [ ]:
# Definir el número de saltos
step = 5

# Agrupar años por rangos de 'step' años
dfc["ApprovalFY_Grouped"] = dfc["ApprovalFY"].apply(lambda year: f"{(year // step) * step}-{(year // step) * step + step - 1}")

# Mostrar conteo por cada grupo de años
print(dfc["ApprovalFY_Grouped"].value_counts())

Para interpretar los datos de una manera más sencilla y poder ver su distribución en función de periodos de años, se ha utilizado un histograma.

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(dfc["ApprovalFY"], bins=range(int(dfc["ApprovalFY"].min()), int(dfc["ApprovalFY"].max()) + step, step),
         color="royalblue", edgecolor="black", alpha=0.75)

plt.title("Distribution of Loan by Fiscal Year", fontsize=14, fontweight='bold')
plt.xlabel("Fiscal Year", fontsize=12)
plt.ylabel("Number of Loan Petition", fontsize=12)

plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

Codificación las agrupaciones para facilitar el entrenamiento para algunos modelos

In [ ]:
dfc["ApprovalFY_Grouped_cod"] = dfc["ApprovalFY_Grouped"].astype('category').cat.codes.astype('int64')
print(dfc["ApprovalFY_Grouped_cod"].value_counts())


##### - NoEmp

Comprobación de todos los valores de la columna *NoEmp*

In [ ]:
dfc['NoEmp'].value_counts()

Histograma de la distribución del número de empleados, para decidir de que manera los agruparemos.

In [ ]:
plt.figure(figsize=(12, 6))

plt.hist(dfc["NoEmp"], bins=range(0, 101, 1), color="royalblue", edgecolor="black", alpha=0.75)

plt.title("Distribution of Number of Employees in Businesses", fontsize=14, fontweight='bold')
plt.xlabel("Number of Employees", fontsize=12)
plt.ylabel("Number of Loan Petition", fontsize=12)

plt.xticks(range(0, 101, 10)) 
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.show()

Como podemos observar, se puede dividir a los diferentes empleados en varios segmentos de grupos, decidiendo.

In [ ]:
# Agrupar número de empleados en las categorías especificadas
bins = [-1, 0, 2, 5, 15, np.inf]
labels = ["0", "1-2", "3-5", "6-15", "15+"]
dfc["NoEmp_Grouped"] = pd.cut(dfc["NoEmp"], bins=bins, labels=labels, right=True)

# Contar la cantidad de cada categoría
category_counts = dfc["NoEmp_Grouped"].value_counts().sort_index()

Visualizar los diferentes grupos creados para comprobar que están más o menos igualados, proporcionando información que puede llegar a ser relevante para los futuros modelos.

In [ ]:
plt.figure(figsize=(8, 5))
category_counts.plot(kind="bar", color="royalblue", edgecolor="black", alpha=0.75)

plt.title("Distribution of Categories of Number of Employees in Businesses", fontsize=14, fontweight="bold")
plt.xlabel("Number of Employees", fontsize=12)
plt.ylabel("Number of Loan Petition", fontsize=12)

plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

Codificación de los grupos para facilitar el entrenanmiento de algunos modelos.

In [ ]:
dfc["NoEmp_Grouped_cod"] = dfc["NoEmp_Grouped"].astype('category').cat.codes.astype('int64')
dfc["NoEmp_Grouped_cod"].value_counts()

##### - CreateJob

Comprobación de todos los valores de la columna *CreateJob*

In [ ]:
dfc['CreateJob'].value_counts()

Histograma de la distribución del número de empleados, para decidir de que manera los agruparemos.

In [ ]:
plt.figure(figsize=(12, 6))
plt.hist(dfc["CreateJob"], bins=range(0, 101, 1), color="royalblue", edgecolor="black", alpha=0.75)

plt.title("Distribution of Jobs Created", fontsize=14, fontweight="bold")
plt.xlabel("Number of Jobs Created", fontsize=12)
plt.ylabel("Number of Loan Petition", fontsize=12)

plt.xticks(range(0, 101, 10))
plt.show()

Al haber una diferencia muy grande entre no crear y crear trabajo lo pasaremos a binario (0-> Trabajo no creaod, 1-> Trabajo Creado)

In [ ]:
# Convertir la columna 'CreateJob' en una variable binaria
dfc["CreateJob_Binary"] = dfc["CreateJob"].apply(lambda x: 0 if x == 0 else 1)

# Mostrar el conteo de cada categoría
dfc["CreateJob_Binary"].value_counts()


##### - RetainedJob

Comprobación de todos los valores de la columna *Retainedjob*

In [ ]:
dfc['RetainedJob'].value_counts()

Histograma de la distribución del número de empleados, para decidir de que manera los agruparemos.

In [ ]:
plt.figure(figsize=(12, 6))

plt.hist(dfc["RetainedJob"], bins=range(0, 101, 1), color="royalblue", edgecolor="black", alpha=0.75)

plt.title("Distribution of Retained Jobs", fontsize=14, fontweight="bold")
plt.xlabel("Number of Jobs Retained", fontsize=12)
plt.ylabel("Number of Loan Petition", fontsize=12)


plt.xticks(range(0, 101, 10)) 
plt.grid(axis="y", linestyle="--", alpha=0.7)  

plt.show()


Al haber una diferencia muy grande entre no mantener o mantener trabajadores lo pasaremos a binario (0-> Trabajo no mantenido, 1-> Trabajo mantenido)

In [ ]:
# Convertir la columna 'CreateJob' en una variable binaria
dfc["RetainedJob_Binary"] = dfc["RetainedJob"].apply(lambda x: 0 if x == 0 else 1)

dfc["RetainedJob_Binary"].value_counts()

##### - DisbursementGross

Comprobación de todos los valores de la columna ***DisbursementGross***

In [ ]:
# print(conteo_valores["DisbursementGross"])
dfc['DisbursementGross'].value_counts()

Formatear los datos de ***DisbursementGross***, ya que al tener las cantidades monetarias el símbolo **'$'**, estarán en formato string. Al quitar este carácter, los podemos pasar a int.

In [ ]:
dfc["DisbursementGross"] = dfc["DisbursementGross"].replace(r'[\$,]', '', regex=True).str.strip().astype(float).astype(int)

dfc["DisbursementGross"].value_counts()

Agrupar las diferentes cantidades de dinero por si pueden llegar a proporcionar información de manera más sencilla.

In [ ]:
# Definir nuevos límites (bins) basados en la distribución analizada
bins = [-1, 30000, 75000, 150000, 500000, np.inf]
labels = ["0-30k", "30k-75k", "75k-150k", "150k-500k", "500k+"]

# Agrupar según los rangos definidos
dfc["DisbursementGross_Grouped"] = pd.cut(dfc["DisbursementGross"], bins=bins, labels=labels, right=True)

# Mostrar el conteo por cada rango
print(dfc["DisbursementGross_Grouped"].value_counts())


Codificaremos los diferentes ***DisbursementGross_Grouped*** para facilitar el entrenamiento en determinados modelos.

In [ ]:
dfc["DisbursementGross_Grouped_cod"] = dfc["DisbursementGross_Grouped"].astype('category').cat.codes.astype('int64')
dfc["DisbursementGross_Grouped_cod"].value_counts()

## 5. Correlacion

Creación de un nuevo ***DataFrame*** a partir del ***DataFrame train*** tratado para no modificar el original.

In [ ]:
dfn = dfc.copy()
dfn.dtypes

Extraer del *dataframe* creado a lo largo del *notebook* solo los parámetros de tipo ***Int***.

In [ ]:
dfn = dfn.select_dtypes(include=[int]) 
dfn.dtypes


Calcular y mostrar matriz de correlación del *dataframe* ***train***

In [ ]:
corrMatrix = dfn.corr()
print(corrMatrix)

Visualización de la matriz de correlación una vez realizado ***One-Hot*** a través de un mapa de calor.

In [ ]:
sns.clustermap(corrMatrix, annot=True, figsize=(20,20))
plt.show()

> Las diferentes caractríticas de interés serán seleccionadas a la hora de realizar los diferentes modelos.

## 6. Codificacion One-Hot 

Realización de One-Hot para el atributo ***BankState_Categorized***

In [ ]:
dfa = dfc.copy() # Backup del dataframe original
dfc = pd.get_dummies(dfc, columns=['BankState_Categorized'], prefix='BankState', dtype=int)
dfc['BankState_Categorized'] = dfa['BankState_Categorized'].values
# Verificar las nuevas columnas creadas
dfc.columns



Visualizar en la tabla de *dataset* el proceso realizado

In [ ]:
dfc.head()

Visualización de la matriz de correlación una vez realizado ***One-Hot*** a través de un mapa de calor.

In [ ]:
dfn = dfc.select_dtypes(include=[int]) 
corrMatrixa = dfn.corr()
sns.clustermap(corrMatrixa, annot=True, figsize=(20, 20)) 
plt.show()

## 7. Create Test

In [ ]:
dfa.dtypes.sort_index()

In [ ]:
df_test = pd.read_csv("data/test_nolabel.csv")

In [ ]:
# Opcional: Eliminar filas con valores nulos en el DataFrame 'df_test'
# df_test = df_test.dropna()

# # Mostrar la cantidad de valores nulos restantes en 'df_test'
print(df_test.isnull().sum())

Eliminar elementos que no queramos del dataframe test, pero aqui si queremos los id para kaggel

In [ ]:
# Eliminar la columna 'id' del DataFrame 'featuresUseless'
featuresUselessWithoutId = featuresUseless.copy()
featuresUselessWithoutId.remove('id')
df_test = df_test.drop(columns=featuresUselessWithoutId)

# Verificar los tipos de datos de las columnas
df_test.dtypes
df_testn = df_test.copy()


Limpiar y estandarizar todos los atributos para que sea igual que el *dataset* ***train***

In [ ]:
# df_testn = df_test.copy()
##############
## Binarias 
##############
### NewExist_Binary                     int64

#   # df_testn = df_testn[df_testn['NewExist'] != 0.0]

df_testn['NewExist_Binary'] = df_testn['NewExist'].replace({
    1.0: 1,   # Existing -> 1
    0.0: 1,   # Existing -> 1
    2.0: 0,   # New -> 0
})

df_testn['NewExist_Binary'] = df_testn['NewExist'].astype('int64')
df_testn = df_testn.drop(columns=['NewExist'])

## Franchise_Binary                    int64
df_testn['Franchise_Binary'] = np.where(df_testn['FranchiseCode'].isin([0, 1]), 0, 1)
df_testn = df_testn.drop(columns=['FranchiseCode'])

### UrbanRural_Binary                   int64
df_testn['UrbanRural_Binary'] = df_testn['UrbanRural'].replace({
    0: 1    
})

df_testn['UrbanRural_Binary'] = df_testn['UrbanRural_Binary'].replace({
    1: 1,
    2: 0    
})
df_testn = df_testn.drop(columns=['UrbanRural'])

### RevLineCr_Binary                    int64
category_map = {'Y': 1, 'T': 1, 'N': 0, '0': 0}

# # # df_testn = df_testn[df_testn['RevLineCr'] != '`']
df_testn['RevLineCr'] = df_testn['RevLineCr'].fillna('N')
df_testn['RevLineCr_Binary'] = df_testn['RevLineCr'].map(category_map).astype(float).astype(int)
df_testn = df_testn.drop(columns=['RevLineCr'])

## LowDoc_Binary                       int64
df_testn['LowDoc'] = df_testn['LowDoc'].fillna('N')
df_testn.loc[~df_testn['LowDoc'].isin(['Y', 'N', '0']), 'LowDoc'] = 'N'# Eliminar valores 'A', 'S', 'R', 'C'
df_testn['LowDoc'] = df_testn['LowDoc'].replace('0', 'N')
df_testn['LowDoc_Binary'] = df_testn['LowDoc'].map({'Y': 1, 'N': 0})
df_testn = df_testn.drop(columns=['LowDoc'])

In [ ]:
# df_testn = df_test.copy()
##############
## Fechas
##############
### ApprovalDate                        int64
### ApprovalDate_quarter                int64
# print(df_testn['ApprovalDate'].isnull().sum())
df_testn['ApprovalDate'] = df_testn['ApprovalDate'].apply(extract_month)
df_testn['ApprovalDate_quarter'] = df_testn['ApprovalDate'].apply(month_to_quarter)

# ### DisbursementDate                    int64
# ### DisbursementDate_quarter            int64
df_testn['DisbursementDate'] = df_testn['DisbursementDate'].apply(lambda x: extract_month(x, mode=4))
df_testn['DisbursementDate_quarter'] = df_testn['DisbursementDate'].apply(month_to_quarter)



In [ ]:
# df_testn = df_test.copy()
##############
## Categorías
##############
### Bank_Categorized                   object
### Bank_Categorized_cod                int64
df_testn['Bank'] = df_testn['Bank'].fillna("THE HUNTINGTON NATIONAL BANK")
categorias_existentes = dfc["Bank_Categorized"].unique()  # Obtener las categorías existentes
df_testn["Bank_Categorized"] = df_testn["Bank"].apply(lambda x: x if x in categorias_existentes else "Otros")
df_testn["Bank_Categorized_cod"] = df_testn["Bank_Categorized"].astype('category').cat.codes.astype('int64')

df_testn = df_testn.drop(columns=['Bank'])


### BankState_Categorized              object
### BankState_Categorized_cod           int64
categorias_existentes_BankState_Categorized = dfc["BankState_Categorized"].unique()  # Obtener las categorías existentes

# BankState
df_testn['BankState'] = df_testn['BankState'].fillna("OH")


def categorize_banksState():
    """
    Asigna categorías a BankState en df_testn basándose en las categorías existentes.
    """
    df_testn["BankState_Categorized"] = df_testn["BankState"].apply(
        lambda x: x if x in categorias_existentes_BankState_Categorized else "Otros"
    )
    return df_testn

df_testn = categorize_banksState()
df_testn["BankState_Categorized_cod"] = df_testn["BankState_Categorized"].astype('category').cat.codes.astype('int64')
df_testn = df_testn.drop(columns=['BankState'])

# #-### BankState_IL
# #-### BankState_DE
# #-### BankState_OH
# #-### BankState_Otros
# #-### BankState_RI
dfk =  df_testn.copy()

df_testn = pd.get_dummies(df_testn, columns=['BankState_Categorized'], prefix='BankState', dtype=int)
df_testn['BankState_Categorized'] = dfk['BankState_Categorized'].values


### ApprovalFY                          int64
### ApprovalFY_Grouped                 object
### ApprovalFY_Grouped_cod              int64
df_testn['ApprovalFY'] = df_testn['ApprovalFY'].astype(str).str.extract(r'(\d{4})')  # Extraer solo los años
df_testn['ApprovalFY'] = df_testn['ApprovalFY'].astype(float).astype(int)  # Convertir a entero
step = 5
df_testn["ApprovalFY_Grouped"] = df_testn["ApprovalFY"].apply(
    lambda year: f"{(year // step) * step}-{(year // step) * step + step - 1}"
)
df_testn["ApprovalFY_Grouped_cod"] = df_testn["ApprovalFY_Grouped"].astype('category').cat.codes.astype('int64')

df_testn["ApprovalFY_Grouped_cod"].value_counts()

### NoEmp                               int64
### NoEmp_Grouped                    category
### NoEmp_Grouped_cod                   int64
bins = [-1, 0, 2, 5, 15, np.inf]
labels = ["0", "1-2", "3-5", "6-15", "15+"]
df_testn["NoEmp_Grouped"] = pd.cut(df_testn["NoEmp"], bins=bins, labels=labels, right=True)
category_counts = df_testn["NoEmp_Grouped"].value_counts().sort_index()

df_testn["NoEmp_Grouped_cod"] = df_testn["NoEmp_Grouped"].astype('category').cat.codes.astype('int64')


### CreateJob                           int64
### CreateJob_Binary                    int64
df_testn["CreateJob_Binary"] = df_testn["CreateJob"].apply(lambda x: 0 if x == 0 else 1)


### RetainedJob                         int64
### RetainedJob_Binary                  int64
df_testn["RetainedJob_Binary"] = df_testn["RetainedJob"].apply(lambda x: 0 if x == 0 else 1)



### DisbursementGross                   int64
### DisbursementGross_Grouped        category
### DisbursementGross_Grouped_cod       int64
df_testn["DisbursementGross"] = df_testn["DisbursementGross"].replace(r'[\$,]', '', regex=True).str.strip().astype(float).astype(int)

bins = [-1, 30000, 75000, 150000, 500000, np.inf]
labels = ["0-30k", "30k-75k", "75k-150k", "150k-500k", "500k+"]
df_testn["DisbursementGross_Grouped"] = pd.cut(df_testn["DisbursementGross"], bins=bins, labels=labels, right=True)
df_testn["DisbursementGross_Grouped_cod"] = df_testn["DisbursementGross_Grouped"].astype('category').cat.codes.astype('int64')

In [ ]:
df_testn.dtypes.sort_index()

In [ ]:
print(df_testn.isnull().sum())


In [ ]:
print(df_testn.shape)
df_testn.info()

## 8. Exportar CSV

Comparar los dos dataset para ver que todo esta bien

In [ ]:
df_check =  pd.read_csv("data/test_nolabel.csv")

print(df_check.shape)
print(df_testn.shape)

In [ ]:
# Comparar las columnas de dfc y df_testn
columns_dfc = set(dfc.columns)
columns_df_testn = set(df_testn.columns)

# Encontrar las diferencias
differences = columns_dfc.symmetric_difference(columns_df_testn)

# Verificar si la única diferencia es 'id'
if differences == {'id', 'Accept'}:
    print("La única diferencia entre las columnas es 'id' y 'Accept'.")
else:
    print("Existen otras diferencias en las columnas:", differences)

Guardar en *formated* los csv train y test

In [ ]:
# Exportar el DataFrame 'dfn' a un archivo CSV
dfc.to_csv('./formated/train_exportado.csv', index=False)

df_testn.to_csv('./formated/test_exportado.csv', index=False)